In [1]:
!pip install pystac_client

  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 2.6 MB/s eta 0:00:00a 0:00:01
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached urllib3-1.26.13-py2.py3-none-any.whl (140 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)


In [3]:
api_url = "https://earth-search.aws.element84.com/v0"

In [4]:
from pystac_client import Client

client = Client.open(api_url)

In [5]:
collection = "sentinel-s2-l2a-cogs"  # Sentinel-2, Level 2A, COGs

In [31]:
from shapely.geometry import Point
point = Point(4.89, 52.37)

In [32]:
search = client.search(
    collections=[collection],
    intersects=point,
    max_items=10,
)

print(search.matched())

744


In [33]:
items = search.get_all_items()

In [9]:
print(len(items))

10


In [10]:
for item in items:
    print(item)

<Item id=S2B_31UFU_20221228_0_L2A>
<Item id=S2A_31UFU_20221226_0_L2A>
<Item id=S2A_31UFU_20221223_1_L2A>
<Item id=S2B_31UFU_20221221_0_L2A>
<Item id=S2B_31UFU_20221218_0_L2A>
<Item id=S2A_31UFU_20221216_0_L2A>
<Item id=S2A_31UFU_20221213_0_L2A>
<Item id=S2B_31UFU_20221211_0_L2A>
<Item id=S2B_31UFU_20221208_0_L2A>
<Item id=S2A_31UFU_20221206_0_L2A>


In [11]:
item = items[0]
print(item.datetime)
print(item.geometry)
print(item.properties)

2022-12-28 10:46:24+00:00
{'type': 'Polygon', 'coordinates': [[[6.071664488869862, 52.22257539160586], [4.819975198242576, 52.24852584329288], [5.083000941495134, 52.85566378666589], [5.245868535710416, 53.22845674062744], [6.1417542968794585, 53.20819279121764], [6.071664488869862, 52.22257539160586]]]}
{'datetime': '2022-12-28T10:46:24Z', 'platform': 'sentinel-2b', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'gsd': 10, 'view:off_nadir': 0, 'proj:epsg': 32631, 'sentinel:utm_zone': 31, 'sentinel:latitude_band': 'U', 'sentinel:grid_square': 'FU', 'sentinel:sequence': '0', 'sentinel:product_id': 'S2B_MSIL2A_20221228T104349_N0509_R008_T31UFU_20221228T120549', 'sentinel:data_coverage': 66.18, 'eo:cloud_cover': 100, 'sentinel:valid_cloud_cover': True, 'sentinel:processing_baseline': '05.09', 'sentinel:boa_offset_applied': True, 'created': '2022-12-28T16:33:31.137Z', 'updated': '2022-12-28T16:33:31.137Z'}


In [16]:
# Exercise: Search satellite scenes using metadata filters

from shapely.affinity import translate

p1 = translate(point, -0.01, -0.01)
p2 = translate(point, 0.01, 0.01)

search = client.search(
    collections=[collection],
    bbox=[p1.x, p1.y, p2.x, p2.y],
    datetime="2020-03-20/2020-03-30",
    query={
          "eo:cloud_cover": 10  
    },
)

In [22]:
items = search.get_all_items()

In [23]:
items

ID: S2A_31UFU_20200328_0_L2A
"Bounding Box: [4.814996623122647, 52.22257539160586, 6.1417542968794585, 53.228577890548]"
Datetime: 2020-03-28 10:46:21+00:00
datetime: 2020-03-28T10:46:21Z
platform: sentinel-2a
constellation: sentinel-2
instruments: ['msi']
gsd: 10
view:off_nadir: 0
proj:epsg: 32631
sentinel:utm_zone: 31


In [24]:
print(len(items)) # Count of scenes available
items.save_object("results.json") # Results in GeoJSON format

4


In [25]:
assets = items[0].assets
print(assets.keys())

dict_keys(['thumbnail', 'overview', 'info', 'metadata', 'visual', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL'])


In [34]:
for key, asset in assets.items():
    print(f"{key}: {asset.title}")

thumbnail: Thumbnail
overview: True color image
info: Original JSON metadata
metadata: Original XML metadata
visual: True color image
B01: Band 1 (coastal)
B02: Band 2 (blue)
B03: Band 3 (green)
B04: Band 4 (red)
B05: Band 5
B06: Band 6
B07: Band 7
B08: Band 8 (nir)
B8A: Band 8A
B09: Band 9
B11: Band 11 (swir16)
B12: Band 12 (swir22)
AOT: Aerosol Optical Thickness (AOT)
WVP: Water Vapour (WVP)
SCL: Scene Classification Map (SCL)


In [35]:
print(assets["thumbnail"].href)

https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/31/U/FU/2020/3/28/0/preview.jpg


In [36]:
import rioxarray
b01_href = assets["B01"].href
b01 = rioxarray.open_rasterio(b01_href)
print(b01)

<xarray.DataArray (band: 1, y: 1830, x: 1830)>
[3348900 values with dtype=uint16]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 6e+05 6.001e+05 6.002e+05 ... 7.097e+05 7.098e+05
  * y            (y) float64 5.9e+06 5.9e+06 5.9e+06 ... 5.79e+06 5.79e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  AVERAGE
    _FillValue:          0
    scale_factor:        1.0
    add_offset:          0.0


In [37]:
b01.rio.to_raster("B01.tif")

In [41]:
# Exercise: Downloading Landsat 8 Assets
url = "https://cmr.earthdata.nasa.gov/stac/LPCLOUD"
collection = "HLSL30.v2.0"
point = Point(-73.97, 40.78)

client = Client.open(url)

search = client.search(
    collections=[collection],
    datetime="2021-02-01/2021-03-31",
    intersects=point
)

items = search.get_all_items()

In [47]:
items[0].assets["browse"].href

'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-public/HLSL30.020/HLS.L30.T18TWL.2021039T153324.v2.0/HLS.L30.T18TWL.2021039T153324.v2.0.jpg'